# Creating a Placement Algorithm

This tutorial demonstrates how we can create a simple placement algorithm on EdgeSimPy.

Let's start by importing the EdgeSimPy modules:

In [1]:
import sys

# Function to check and install required packages
def install_edgesimpy():
    try:
        # Try importing EdgeSimPy and dependencies
        from edge_sim_py import *
        import networkx as nx
        import msgpack
        import matplotlib.pyplot as plt
        import pandas as pd
        import numpy as np
        print("✅ EdgeSimPy and dependencies are already installed.")
    except ModuleNotFoundError:
        print("⚠️ EdgeSimPy not found. Installing now...")

        # Fix dependency conflicts before installation
        !pip uninstall -y networkx msgpack numpy pandas matplotlib
        !pip install -q git+https://github.com/EdgeSimPy/EdgeSimPy.git@v1.1.0

        # Reinstall the correct versions of required dependencies
        !pip install -q networkx==3.2 msgpack numpy pandas matplotlib

        # Refresh imports after installation
        try:
            from edge_sim_py import *
            import networkx as nx
            import msgpack
            import matplotlib.pyplot as plt
            import pandas as pd
            import numpy as np
            print("✅ EdgeSimPy and dependencies have been successfully installed!")
        except ModuleNotFoundError as e:
            print("❌ Installation failed:", e)

# Run installation function
install_edgesimpy()


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.1 requires networkx>=3.0, but you have networkx 2.6.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompa

## Implementing the Placement Algorithm

In this example, we are going to create a simple placement algorithm that works according to the well-known First-Fit heuristic. In a nutshell, our algorithm will provision each service to the first edge server with available resources to host them.

In [ ]:
def my_algorithm(parameters):
    # We can always call the 'all()' method to get a list with all created instances of a given class
    for service in Service.all():
        # We don't want to migrate services are are already being migrated
        if service.server == None and not service.being_provisioned:

            # Let's iterate over the list of edge servers to find a suitable host for our service
            for edge_server in EdgeServer.all():

                # We must check if the edge server has enough resources to host the service
                if edge_server.has_capacity_to_host(service=service):

                    # Start provisioning the service in the edge server
                    service.provision(target_server=edge_server)

                    # After start migrating the service we can move on to the next service
                    break

## Running the Simulation

As we're creating a placement algorithm, we must instruct EdgeSimPy that it needs to continue the simulation until all services are provisioned within the infrastructure.

To do so, let's create a simple function that will be used as the simulation's stopping criterion. EdgeSimPy will run that function at the end of each time step, halting the simulation as soon as it returns `True`.

In [ ]:
def stopping_criterion(model: object):
    # Defining a variable that will help us to count the number of services successfully provisioned within the infrastructure
    provisioned_services = 0

    # Iterating over the list of services to count the number of services provisioned within the infrastructure
    for service in Service.all():

        # Initially, services are not hosted by any server (i.e., their "server" attribute is None).
        # Once that value changes, we know that it has been successfully provisioned inside an edge server.
        if service.server != None:
            provisioned_services += 1

    # As EdgeSimPy will halt the simulation whenever this function returns True, its output will be a boolean expression
    # that checks if the number of provisioned services equals to the number of services spawned in our simulation
    return provisioned_services == Service.count()

Once we have our stopping criterion, we can finally run our simulation by creating an instance of the `Simulator` class, loading a dataset, and calling the `run_model()` method.

In [ ]:

# Creating a Simulator object
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset from GitHub
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

# Executing the simulation
simulator.run_model()

# Checking the placement output
for service in Service.all():
    print(f"{service}. Host: {service.server}")